In [4]:
%%capture
!pip install wandb -qqq

In [5]:
import tensorflow.keras as keras
from google.colab import drive
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback

MODELS_PATH = "/content/gdrive/My Drive/magshimim/camerona/models/"
BASE = "gender_base_model.h5"
BEST = "gender_best_model.h5"
DATASET_PATH_DRIVE = "/content/gdrive/My Drive/magshimim/camerona/dataset/gender/"
DATASET_PATH_COLAB = "/content/gender/"
VAL_TEXT = DATASET_PATH_COLAB+"val.txt"
TRAIN_TEXT = DATASET_PATH_COLAB+"train.txt"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"

drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g6OeZ3DZMlDwMzaBGrvPzs41HKylKdCTSG3HcSjwkPoqn148GtR5NM
Mounted at /content/gdrive/


In [6]:
 !ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/gender/" "/content/"

In [7]:
 !wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
def buildModel(lr = 0.001):
  input_layer = keras.layers.Input(shape=[64,64,3])

  #block1
  layer = keras.layers.Conv2D(32, (3,3), strides=(1,1), activation='relu', padding='same', name='block1_conv1')(input_layer)
  layer = keras.layers.MaxPool2D(2, name='block1_max1')(layer)

  #block2
  layer = keras.layers.Conv2D(16, (3,3), strides=(1,1), activation='relu', padding='same', name='block2_conv1')(layer)
  layer = keras.layers.MaxPool2D(2, name='block2_max1')(layer)

  #output block
  layer = keras.layers.Flatten()(layer)
  layer = keras.layers.Dense(16, activation="relu")(layer)
  layer = keras.layers.Dense(1, activation="sigmoid", name='output_layer')(layer)
 
  model = keras.Model(inputs=input_layer, outputs=layer)
  opt = keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer = opt, loss = ['binary_crossentropy'], metrics=[keras.metrics.BinaryAccuracy()])
  print(model.summary())

  return model

In [9]:
def getData(file_path):
  f = open(file_path, 'r')
  data = f.read().split('\n')
  f.close()
  x = []
  y = []
  all = len(data)
  cnt = 1
  for line in data:
    if cnt%250 == 0:
      print("{0}/{1}".format(cnt, all))
    cnt += 1
    line_data = line.split(" ")
    line_data[0] = line_data[0].replace('\\', '/')
    try:
      img = cv2.imread(DATASET_PATH_COLAB+line_data[0])
      img = cv2.resize(img, (64,64))/255
    except:
      continue
    x.append(img)
    y.append(int(line_data[1]))
  x = np.array(x)
  y = np.array(y)
  return x,y

In [10]:
#building and saving base model
model = buildModel()
model.save(MODELS_PATH+BASE)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 32)        896       
_________________________________________________________________
block1_max1 (MaxPooling2D)   (None, 32, 32, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        4624      
_________________________________________________________________
block2_max1 (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                65552 

In [11]:
wandb.init(project="gender-classification")

wandb: Currently logged in as: ofekh (use `wandb login --relogin` to force relogin)


In [12]:
#callbacks
best_callback = keras.callbacks.ModelCheckpoint(MODELS_PATH+BEST, save_best_only=True)
wandb_callback = WandbCallback()

In [13]:
#loading data
print("loading validation data")
val_x, val_y = getData(VAL_TEXT)
print("loading train data")
train_x, train_y = getData(TRAIN_TEXT)

loading validation data
250/1765
500/1765
750/1765
1000/1765
1250/1765
1500/1765
1750/1765
loading train data
250/13241
500/13241
750/13241
1000/13241
1250/13241
1500/13241
1750/13241
2000/13241
2250/13241
2500/13241
2750/13241
3000/13241
3250/13241
3500/13241
3750/13241
4000/13241
4250/13241
4500/13241
4750/13241
5000/13241
5250/13241
5500/13241
5750/13241
6000/13241
6250/13241
6500/13241
6750/13241
7000/13241
7250/13241
7500/13241
7750/13241
8000/13241
8250/13241
8500/13241
8750/13241
9000/13241
9250/13241
9500/13241
9750/13241
10000/13241
10250/13241
10500/13241
10750/13241
11000/13241
11250/13241
11500/13241
11750/13241
12000/13241
12250/13241
12500/13241
12750/13241
13000/13241


In [15]:
model = buildModel(0.001)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 32)        896       
_________________________________________________________________
block1_max1 (MaxPooling2D)   (None, 32, 32, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        4624      
_________________________________________________________________
block2_max1 (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                6555

In [16]:
model.fit(train_x,
          train_y,
          epochs=24, 
          validation_data=(val_x, val_y),
          callbacks=[best_callback, wandb_callback])

Epoch 1/24
414/414 [==============================] - 55s 131ms/step - loss: 0.5161 - binary_accuracy: 0.7339 - val_loss: 0.3026 - val_binary_accuracy: 0.8924
Epoch 2/24
414/414 [==============================] - 52s 126ms/step - loss: 0.2375 - binary_accuracy: 0.9131 - val_loss: 0.2088 - val_binary_accuracy: 0.9235
Epoch 3/24
414/414 [==============================] - 53s 127ms/step - loss: 0.1648 - binary_accuracy: 0.9363 - val_loss: 0.1920 - val_binary_accuracy: 0.9297
Epoch 4/24
414/414 [==============================] - 52s 127ms/step - loss: 0.1335 - binary_accuracy: 0.9482 - val_loss: 0.1385 - val_binary_accuracy: 0.9456
Epoch 5/24
414/414 [==============================] - 53s 129ms/step - loss: 0.0990 - binary_accuracy: 0.9608 - val_loss: 0.1225 - val_binary_accuracy: 0.9535
Epoch 6/24
414/414 [==============================] - 52s 127ms/step - loss: 0.0821 - binary_accuracy: 0.9686 - val_loss: 0.1332 - val_binary_accuracy: 0.9501
Epoch 7/24
414/414 [==========================

In [17]:
test_x, test_y = getData(TEST_TEXT)

250/2648
500/2648
750/2648
1000/2648
1250/2648
1500/2648
1750/2648
2000/2648
2250/2648
2500/2648


In [18]:
loss, acc = model.evaluate(x = test_x, y = test_y)

83/83 [==============================] - 3s 34ms/step - loss: 0.0217 - binary_accuracy: 0.9943


In [19]:
model.save(MODELS_PATH+BEST)